<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/ML%20confusion%20matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkkgYW0gYSBzdHVkZW50IGxlYXJuaW5nIE1MLiBJIGhhdmUgYSB0cmFpbmVkIGEgbW9kZWwgdG8gcHJlZGljdCB3aGV0aGVyIGFuIGltYWdlIGlzIGEgY2F0LCBkb2csIG9yIHNxdWlycmVsLiBUaGVzZSBhcmUgdGhlIHByZWRpY3Rpb25zIEkmIzM5O3ZlIGdvdHRlbiBmcm9tIG15IG1vZGVsLCBhbmQgdGhlIGdyb3VuZCB0cnV0aCBmcm9tIG15IGxhYmVscy4gSGVyZSwgMCBjb3JyZXNwb25kcyB0byBjYXQsIDEgdG8gZG9nLCBhbmQgMiB0byBzcXVpcnJlbC5cXG5cXG5wcmVkaWN0aW9ucyA9IFswLCAwLCAxLCAxLCAwLCAwLCAyLCAyLCAxLCAwXVxcbmdyb3VuZF90cnV0aCA9IFswLCAxLCAxLCAxLCAxLCAwLCAyLCAyLCAyLCAyXVxcblxcbkkgd291bGQgbGlrZSB0byBtYWtlIGEgY29uZnVzaW9uIG1hdHJpeCB1c2luZyBza2xlYXJuLiBJIHdvdWxkIGFsc28gbGlrZSB0byBvdXRwdXQgbWV0cmljcyBsaWtlIHByZWNpc2lvbiwgcmVjYWxsLCBhbmQgYWNjdXJhY3kuIEFuZCBJIHdvdWxkIGxpa2UgdG8gdmlzdWFsaXplIHRoZSBjb25mdXNpb24gbWF0cml4IGJ5IHBsb3R0aW5nIGl0LiBUaGUgcGxvdCBzaG91bGQgc2hvdyB0aGUgY2xhc3MgbmFtZXMgaW5zdGVhZCBvZiB0aGUgbnVtZXJpYyBsYWJlbHMuXFxuXFxuQ2FuIHlvdSBnZW5lcmF0ZSBjb2RlIHRvIGRvIHRoaXMgZm9yIG1lIGluIFB5dGhvbj8ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJIZXJlJ3MgYSBQeXRob24gc29sdXRpb24gdXNpbmcgc2Npa2l0LWxlYXJuIChza2xlYXJuKSB0byBjcmVhdGUgYSBjb25mdXNpb24gbWF0cml4LCBjYWxjdWxhdGUgcHJlY2lzaW9uLCByZWNhbGwsIGFuZCBhY2N1cmFjeSwgYW5kIHZpc3VhbGl6ZSB0aGUgY29uZnVzaW9uIG1hdHJpeCB3aXRoIGNsYXNzIG5hbWVzLlxcblxcbmBgYHB5dGhvblxcbmltcG9ydCBudW1weSBhcyBucFxcbmltcG9ydCBtYXRwbG90bGliLnB5cGxvdCBhcyBwbHRcXG5mcm9tIHNrbGVhcm4ubWV0cmljcyBpbXBvcnQgY29uZnVzaW9uX21hdHJpeCwgY2xhc3NpZmljYXRpb25fcmVwb3J0LCBhY2N1cmFjeV9zY29yZVxcbmltcG9ydCBzZWFib3JuIGFzIHNuc1xcblxcbnByZWRpY3Rpb25zID0gbnAuYXJyYXkoWzAsIDAsIDEsIDEsIDAsIDAsIDIsIDIsIDEsIDBdKVxcbmdyb3VuZF90cnV0aCA9IG5wLmFycmF5KFswLCAxLCAxLCAxLCAxLCAwLCAyLCAyLCAyLCAyXSlcXG5jbGFzc19uYW1lcyA9IFtcXFwiY2F0XFxcIiwgXFxcImRvZ1xcXCIsIFxcXCJzcXVpcnJlbFxcXCJdXFxuXFxuXFxuIyBDYWxjdWxhdGUgdGhlIGNvbmZ1c2lvbiBtYXRyaXhcXG5jbSA9IGNvbmZ1c2lvbl9tYXRyaXgoZ3JvdW5kX3RydXRoLCBwcmVkaWN0aW9ucylcXG5cXG4jIENhbGN1bGF0ZSBhY2N1cmFjeSwgcHJlY2lzaW9uIGFuZCByZWNhbGxcXG5hY2N1cmFjeSA9IGFjY3VyYWN5X3Njb3JlKGdyb3VuZF90cnV0aCwgcHJlZGljdGlvbnMpXFxucmVwb3J0ID0gY2xhc3NpZmljYXRpb25fcmVwb3J0KGdyb3VuZF90cnV0aCwgcHJlZGljdGlvbnMsIHRhcmdldF9uYW1lcz1jbGFzc19uYW1lcylcXG5cXG5cXG4jIFZpc3VhbGl6ZSB0aGUgY29uZnVzaW9uIG1hdHJpeFxcbnBsdC5maWd1cmUoZmlnc2l6ZT0oOCwgNikpXFxuc25zLmhlYXRtYXAoY20sIGFubm90PVRydWUsIGZtdD1cXFwiZFxcXCIsIGNtYXA9XFxcIkJsdWVzXFxcIiwgXFxuICAgICAgICAgICAgeHRpY2tsYWJlbHM9Y2xhc3NfbmFtZXMsIHl0aWNrbGFiZWxzPWNsYXNzX25hbWVzKVxcbnBsdC54bGFiZWwoXFxcIlByZWRpY3RlZFxcXCIpXFxucGx0LnlsYWJlbChcXFwiVHJ1ZVxcXCIpXFxucGx0LnRpdGxlKFxcXCJDb25mdXNpb24gTWF0cml4XFxcIilcXG5wbHQuc2hvdygpXFxuXFxuIyBQcmludCB0aGUgY2xhc3NpZmljYXRpb24gcmVwb3J0IFxcbnByaW50KFxcXCJDbGFzc2lmaWNhdGlvbiBSZXBvcnQ6XFxcXG5cXFwiLCByZXBvcnQpXFxucHJpbnQoXFxcIkFjY3VyYWN5OlxcXCIsIGFjY3VyYWN5KVxcblxcbmBgYFxcblxcblRoaXMgY29kZSBmaXJzdCBjYWxjdWxhdGVzIHRoZSBjb25mdXNpb24gbWF0cml4IHVzaW5nIGBjb25mdXNpb25fbWF0cml4YC4gIFRoZW4gaXQgY29tcHV0ZXMgIHRoZSBhY2N1cmFjeSB1c2luZyBgYWNjdXJhY3lfc2NvcmVgIGFuZCBhIGNsYXNzaWZpY2F0aW9uIHJlcG9ydCAoaW5jbHVkaW5nIHByZWNpc2lvbiBhbmQgcmVjYWxsKSB1c2luZyBgY2xhc3NpZmljYXRpb25fcmVwb3J0YC4gRmluYWxseSwgaXQgdXNlcyBgc2VhYm9ybmAgKG1ha2Ugc3VyZSB5b3UgaGF2ZSBgc2VhYm9ybmAgaW5zdGFsbGVkOiBgcGlwIGluc3RhbGwgc2VhYm9ybmApIHRvIGNyZWF0ZSBhIGhlYXRtYXAgdmlzdWFsaXphdGlvbiBvZiB0aGUgY29uZnVzaW9uIG1hdHJpeCB3aXRoIHRoZSBjbGFzcyBuYW1lcyBjbGVhcmx5IGxhYmVsZWQuICBUaGUgY2xhc3NpZmljYXRpb24gcmVwb3J0IGFuZCBhY2N1cmFjeSBhcmUgdGhlbiBwcmludGVkIHRvIHRoZSBjb25zb2xlLiBSZW1lbWJlciB0byBpbnN0YWxsIHRoZSBuZWNlc3NhcnkgbGlicmFyaWVzIChgc2Npa2l0LWxlYXJuYCwgYG1hdHBsb3RsaWJgLCBgc2VhYm9ybmApIGlmIHlvdSBoYXZlbid0IGFscmVhZHkuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MiwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "I am a student learning ML. I have a trained a model to predict whether an image is a cat, dog, or squirrel. These are the predictions I&#39;ve gotten from my model, and the ground truth from my labels. Here, 0 corresponds to cat, 1 to dog, and 2 to squirrel.\\n\\npredictions = [0, 0, 1, 1, 0, 0, 2, 2, 1, 0]\\nground_truth = [0, 1, 1, 1, 1, 0, 2, 2, 2, 2]\\n\\nI would like to make a confusion matrix using sklearn. I would also like to output metrics like precision, recall, and accuracy. And I would like to visualize the confusion matrix by plotting it. The plot should show the class names instead of the numeric labels.\\n\\nCan you generate code to do this for me in Python?"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "Here's a Python solution using scikit-learn (sklearn) to create a confusion matrix, calculate precision, r

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

if response.candidates:
    display(Markdown(response.text))
else:
    print("The model did not return any candidates. Please check your prompt and configuration.")
    # Optionally, you can print more details about the response:
    print(f"Response details: {response}")

The model did not return any candidates. Please check your prompt and configuration.
Response details: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "usage_metadata": {
        "prompt_token_count": 713,
        "total_token_count": 713
      }
    }),
)


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

I am a student learning ML. I have a trained a model to predict whether an image is a cat, dog, or squirrel. These are the predictions I&#39;ve gotten from my model, and the ground truth from my labels. Here, 0 corresponds to cat, 1 to dog, and 2 to squirrel.\n\npredictions = [0, 0, 1, 1, 0, 0, 2, 2, 1, 0]\nground_truth = [0, 1, 1, 1, 1, 0, 2, 2, 2, 2]\n\nI would like to make a confusion matrix using sklearn. I would also like to output metrics like precision, recall, and accuracy. And I would like to visualize the confusion matrix by plotting it. The plot should show the class names instead of the numeric labels.\n\nCan you generate code to do this for me in Python? 

-------------------------------------------------------------------------------- 

Role: model 

Here's a Python solution using scikit-learn (sklearn) to create a confusion matrix, calculate precision, recall, and accuracy, and visualize the confusion matrix with class names.\n\n```python\nimport numpy as np